In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField
from pyspark.sql.types import StructType
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lit, count, split, avg

In [2]:
# Create a spark session
spark = SparkSession.builder.appName("Paytm_weather").master("local[*]").getOrCreate()

In [3]:
data_path = "/FileStore/tables/data/*"
df_weather = spark.read.csv(data_path)
df_weather.take(10)

Out[119]: [Row(_c0='STN---', _c1='WBAN', _c2='YEARMODA', _c3='TEMP', _c4='DEWP', _c5='SLP', _c6='STP', _c7='VISIB', _c8='WDSP', _c9='MXSPD', _c10='GUST', _c11='MAX', _c12='MIN', _c13='PRCP', _c14='SNDP', _c15='FRSHTT'),
 Row(_c0='010260', _c1='99999', _c2='20190101', _c3='26.1', _c4='21.2', _c5='1001.9', _c6='987.5', _c7='20.6', _c8='9.0', _c9='15.9', _c10='29.7', _c11='29.8', _c12='21.7*', _c13='0.02G', _c14='18.5', _c15='001000'),
 Row(_c0='010260', _c1='99999', _c2='20190102', _c3='24.9', _c4='22.1', _c5='1020.1', _c6='1005.5', _c7='5.4', _c8='5.6', _c9='13.6', _c10='22.1', _c11='27.1*', _c12='20.7', _c13='0.48G', _c14='22.8', _c15='001000'),
 Row(_c0='010260', _c1='99999', _c2='20190103', _c3='31.7', _c4='29.1', _c5='1008.9', _c6='994.7', _c7='13.6', _c8='11.6', _c9='21.4', _c10='49.5', _c11='37.4*', _c12='26.8*', _c13='0.25G', _c14='999.9', _c15='011000'),
 Row(_c0='010260', _c1='99999', _c2='20190104', _c3='32.9', _c4='30.3', _c5='1011.4', _c6='997.1', _c7='15.8', _c8='4.9', _c9='7.8', _c10='10.9', _c11='36.1', _c12='31.8', _c13='0.52G', _c14='999.9', _c15='001000'),
 Row(_c0='010260', _c1='99999', _c2='20190105', _c3='35.5', _c4='33.0', _c5='1015.7', _c6='1001.4', _c7='12.0', _c8='10.4', _c9='13.6', _c10='21.0', _c11='38.5*', _c12='32.7', _c13='0.02G', _c14='23.6', _c15='010000'),
 Row(_c0='010260', _c1='99999', _c2='20190106', _c3='38.5', _c4='34.1', _c5='1008.2', _c6='994.2', _c7='12.8', _c8='10.0', _c9='17.5', _c10='28.9', _c11='41.4', _c12='33.8*', _c13='0.12G', _c14='23.2', _c15='010000'),
 Row(_c0='010260', _c1='99999', _c2='20190107', _c3='32.1', _c4='29.8', _c5='996.8', _c6='982.7', _c7='6.9', _c8='11.3', _c9='15.5', _c10='28.6', _c11='35.1*', _c12='30.4', _c13='0.00G', _c14='999.9', _c15='001000'),
 Row(_c0='010260', _c1='99999', _c2='20190108', _c3='31.6', _c4='28.0', _c5='997.4', _c6='983.3', _c7='22.9', _c8='5.9', _c9='11.7', _c10='19.0', _c11='34.3', _c12='28.0*', _c13='0.53G', _c14='0.4', _c15='011000'),
 Row(_c0='010260', _c1='99999', _c2='20190109', _c3='29.9', _c4='27.7', _c5='1011.6', _c6='997.3', _c7='29.8', _c8='7.6', _c9='15.2', _c10='26.6', _c11='32.4', _c12='26.1', _c13='0.20G', _c14='23.6', _c15='001000')]

In [4]:
# define schema
schema = StructType(
    [
        StructField("STN_NO", IntegerType(), True),
        StructField("WBAN", IntegerType(), True),
        StructField("YEARMODA", IntegerType(), True),
        StructField("TEMP", FloatType(), True),
        StructField("DEWP", FloatType(), True),
        StructField("SLP", FloatType(), True),
        StructField("STP", FloatType(), True),
        StructField("VISIB", FloatType(), True),
        StructField("WDSP", FloatType(), True),
        StructField("MXSPD", FloatType(), True),
        StructField("GUST", FloatType(), True),
        StructField("MAX", FloatType(), True),
        StructField("MIN", FloatType(), True),
        StructField("PRCP", FloatType(), True),
        StructField("SNDP", FloatType(), True),
        StructField("FRSHTT", StringType(), True),
    ]
)

In [5]:
df_weather = (
    spark.read.schema(schema)
    .option("header","true")
    .csv(
       data_path
    )
)

df_weather.take(10)

Out[121]: [Row(STN_NO=10260, WBAN=99999, YEARMODA=20190101, TEMP=26.100000381469727, DEWP=21.200000762939453, SLP=1001.9000244140625, STP=987.5, VISIB=20.600000381469727, WDSP=9.0, MXSPD=15.899999618530273, GUST=29.700000762939453, MAX=29.799999237060547, MIN=None, PRCP=None, SNDP=18.5, FRSHTT='001000'),
 Row(STN_NO=10260, WBAN=99999, YEARMODA=20190102, TEMP=24.899999618530273, DEWP=22.100000381469727, SLP=1020.0999755859375, STP=1005.5, VISIB=5.400000095367432, WDSP=5.599999904632568, MXSPD=13.600000381469727, GUST=22.100000381469727, MAX=None, MIN=20.700000762939453, PRCP=None, SNDP=22.799999237060547, FRSHTT='001000'),
 Row(STN_NO=10260, WBAN=99999, YEARMODA=20190103, TEMP=31.700000762939453, DEWP=29.100000381469727, SLP=1008.9000244140625, STP=994.7000122070312, VISIB=13.600000381469727, WDSP=11.600000381469727, MXSPD=21.399999618530273, GUST=49.5, MAX=None, MIN=None, PRCP=None, SNDP=999.9000244140625, FRSHTT='011000'),
 Row(STN_NO=10260, WBAN=99999, YEARMODA=20190104, TEMP=32.900001525878906, DEWP=30.299999237060547, SLP=1011.4000244140625, STP=997.0999755859375, VISIB=15.800000190734863, WDSP=4.900000095367432, MXSPD=7.800000190734863, GUST=10.899999618530273, MAX=36.099998474121094, MIN=31.799999237060547, PRCP=None, SNDP=999.9000244140625, FRSHTT='001000'),
 Row(STN_NO=10260, WBAN=99999, YEARMODA=20190105, TEMP=35.5, DEWP=33.0, SLP=1015.7000122070312, STP=1001.4000244140625, VISIB=12.0, WDSP=10.399999618530273, MXSPD=13.600000381469727, GUST=21.0, MAX=None, MIN=32.70000076293945, PRCP=None, SNDP=23.600000381469727, FRSHTT='010000'),
 Row(STN_NO=10260, WBAN=99999, YEARMODA=20190106, TEMP=38.5, DEWP=34.099998474121094, SLP=1008.2000122070312, STP=994.2000122070312, VISIB=12.800000190734863, WDSP=10.0, MXSPD=17.5, GUST=28.899999618530273, MAX=41.400001525878906, MIN=None, PRCP=None, SNDP=23.200000762939453, FRSHTT='010000'),
 Row(STN_NO=10260, WBAN=99999, YEARMODA=20190107, TEMP=32.099998474121094, DEWP=29.799999237060547, SLP=996.7999877929688, STP=982.7000122070312, VISIB=6.900000095367432, WDSP=11.300000190734863, MXSPD=15.5, GUST=28.600000381469727, MAX=None, MIN=30.399999618530273, PRCP=None, SNDP=999.9000244140625, FRSHTT='001000'),
 Row(STN_NO=10260, WBAN=99999, YEARMODA=20190108, TEMP=31.600000381469727, DEWP=28.0, SLP=997.4000244140625, STP=983.2999877929688, VISIB=22.899999618530273, WDSP=5.900000095367432, MXSPD=11.699999809265137, GUST=19.0, MAX=34.29999923706055, MIN=None, PRCP=None, SNDP=0.4000000059604645, FRSHTT='011000'),
 Row(STN_NO=10260, WBAN=99999, YEARMODA=20190109, TEMP=29.899999618530273, DEWP=27.700000762939453, SLP=1011.5999755859375, STP=997.2999877929688, VISIB=29.799999237060547, WDSP=7.599999904632568, MXSPD=15.199999809265137, GUST=26.600000381469727, MAX=32.400001525878906, MIN=26.100000381469727, PRCP=None, SNDP=23.600000381469727, FRSHTT='001000'),
 Row(STN_NO=10260, WBAN=99999, YEARMODA=20190110, TEMP=33.099998474121094, DEWP=30.600000381469727, SLP=979.0999755859375, STP=965.2999877929688, VISIB=5.300000190734863, WDSP=17.799999237060547, MXSPD=24.899999618530273, GUST=41.79999923706055, MAX=41.400001525878906, MIN=None, PRCP=None, SNDP=999.9000244140625, FRSHTT='011000')]

In [6]:
country_schema = StructType(
    [
       StructField("COUNTRY_ABBR", StringType(), True),
       StructField("COUNTRY_FULL", StringType(), True)
    ]
)


df_country = (
    spark.read.schema(country_schema)
    .option("header","true")
    .csv(
       "/FileStore/tables/countrylist.csv"
    )
)

df_country.take(10)

Out[122]: [Row(COUNTRY_ABBR='AA', COUNTRY_FULL='ARUBA'),
 Row(COUNTRY_ABBR='AC', COUNTRY_FULL='ANTIGUA AND BARBUDA'),
 Row(COUNTRY_ABBR='AF', COUNTRY_FULL='AFGHANISTAN'),
 Row(COUNTRY_ABBR='AG', COUNTRY_FULL='ALGERIA'),
 Row(COUNTRY_ABBR='AI', COUNTRY_FULL='ASCENSION ISLAND'),
 Row(COUNTRY_ABBR='AJ', COUNTRY_FULL='AZERBAIJAN'),
 Row(COUNTRY_ABBR='AL', COUNTRY_FULL='ALBANIA'),
 Row(COUNTRY_ABBR='AM', COUNTRY_FULL='ARMENIA'),
 Row(COUNTRY_ABBR='AN', COUNTRY_FULL='ANDORRA'),
 Row(COUNTRY_ABBR='AO', COUNTRY_FULL='ANGOLA')]

In [7]:
station_schema = StructType(
    [
        StructField("STN_NO", StringType(), True),
       StructField("COUNTRY_ABBR", StringType(), True)
    ]
)

df_stations = (
    spark.read.schema(station_schema)
    .option("header","true")
    .csv(
       "/FileStore/tables/stationlist.csv"
    )
)

df_stations.take(10)

Out[123]: [Row(STN_NO='012240', COUNTRY_ABBR='NO'),
 Row(STN_NO='020690', COUNTRY_ABBR='SW'),
 Row(STN_NO='020870', COUNTRY_ABBR='SW'),
 Row(STN_NO='021190', COUNTRY_ABBR='SW'),
 Row(STN_NO='032690', COUNTRY_ABBR='UK'),
 Row(STN_NO='033450', COUNTRY_ABBR='UK'),
 Row(STN_NO='039290', COUNTRY_ABBR='UK'),
 Row(STN_NO='039790', COUNTRY_ABBR='EI'),
 Row(STN_NO='040480', COUNTRY_ABBR='IC'),
 Row(STN_NO='041300', COUNTRY_ABBR='IC')]

In [8]:
df_country_station = df_stations.join(df_country, on=['COUNTRY_ABBR'], how='left')

In [9]:
df_country_station.take(10)

Out[125]: [Row(COUNTRY_ABBR='NO', STN_NO='012240', COUNTRY_FULL='NORWAY'),
 Row(COUNTRY_ABBR='SW', STN_NO='020690', COUNTRY_FULL='SWEDEN'),
 Row(COUNTRY_ABBR='SW', STN_NO='020870', COUNTRY_FULL='SWEDEN'),
 Row(COUNTRY_ABBR='SW', STN_NO='021190', COUNTRY_FULL='SWEDEN'),
 Row(COUNTRY_ABBR='UK', STN_NO='032690', COUNTRY_FULL='UNITED KINGDOM'),
 Row(COUNTRY_ABBR='UK', STN_NO='033450', COUNTRY_FULL='UNITED KINGDOM'),
 Row(COUNTRY_ABBR='UK', STN_NO='039290', COUNTRY_FULL='UNITED KINGDOM'),
 Row(COUNTRY_ABBR='EI', STN_NO='039790', COUNTRY_FULL='IRELAND'),
 Row(COUNTRY_ABBR='IC', STN_NO='040480', COUNTRY_FULL='ICELAND'),
 Row(COUNTRY_ABBR='IC', STN_NO='041300', COUNTRY_FULL='ICELAND')]

In [10]:
df = df_weather.join(df_country_station, on=['STN_NO'], how='left')

In [11]:
df.take(10)

Out[127]: [Row(STN_NO=13840, WBAN=99999, YEARMODA=20190101, TEMP=38.599998474121094, DEWP=24.799999237060547, SLP=9999.900390625, STP=9999.900390625, VISIB=999.9000244140625, WDSP=11.699999809265137, MXSPD=33.599998474121094, GUST=44.099998474121094, MAX=None, MIN=None, PRCP=None, SNDP=999.9000244140625, FRSHTT='000000', COUNTRY_ABBR='NO', COUNTRY_FULL='NORWAY'),
 Row(STN_NO=13840, WBAN=99999, YEARMODA=20190102, TEMP=30.5, DEWP=16.600000381469727, SLP=9999.900390625, STP=9999.900390625, VISIB=999.9000244140625, WDSP=8.0, MXSPD=23.700000762939453, GUST=35.0, MAX=None, MIN=None, PRCP=None, SNDP=999.9000244140625, FRSHTT='000000', COUNTRY_ABBR='NO', COUNTRY_FULL='NORWAY'),
 Row(STN_NO=13840, WBAN=99999, YEARMODA=20190103, TEMP=32.20000076293945, DEWP=21.700000762939453, SLP=9999.900390625, STP=9999.900390625, VISIB=999.9000244140625, WDSP=3.5, MXSPD=13.0, GUST=999.9000244140625, MAX=None, MIN=None, PRCP=None, SNDP=999.9000244140625, FRSHTT='000000', COUNTRY_ABBR='NO', COUNTRY_FULL='NORWAY'),
 Row(STN_NO=13840, WBAN=99999, YEARMODA=20190104, TEMP=34.20000076293945, DEWP=27.200000762939453, SLP=9999.900390625, STP=9999.900390625, VISIB=999.9000244140625, WDSP=2.5999999046325684, MXSPD=6.0, GUST=999.9000244140625, MAX=None, MIN=None, PRCP=None, SNDP=999.9000244140625, FRSHTT='000000', COUNTRY_ABBR='NO', COUNTRY_FULL='NORWAY'),
 Row(STN_NO=13840, WBAN=99999, YEARMODA=20190105, TEMP=30.600000381469727, DEWP=27.399999618530273, SLP=9999.900390625, STP=9999.900390625, VISIB=15.5, WDSP=2.0, MXSPD=6.400000095367432, GUST=999.9000244140625, MAX=None, MIN=None, PRCP=None, SNDP=999.9000244140625, FRSHTT='100000', COUNTRY_ABBR='NO', COUNTRY_FULL='NORWAY'),
 Row(STN_NO=13840, WBAN=99999, YEARMODA=20190106, TEMP=26.600000381469727, DEWP=25.100000381469727, SLP=9999.900390625, STP=9999.900390625, VISIB=8.199999809265137, WDSP=0.699999988079071, MXSPD=4.099999904632568, GUST=999.9000244140625, MAX=None, MIN=None, PRCP=None, SNDP=999.9000244140625, FRSHTT='100000', COUNTRY_ABBR='NO', COUNTRY_FULL='NORWAY'),
 Row(STN_NO=13840, WBAN=99999, YEARMODA=20190107, TEMP=26.200000762939453, DEWP=25.200000762939453, SLP=9999.900390625, STP=9999.900390625, VISIB=2.700000047683716, WDSP=1.7999999523162842, MXSPD=7.0, GUST=999.9000244140625, MAX=None, MIN=None, PRCP=None, SNDP=999.9000244140625, FRSHTT='101000', COUNTRY_ABBR='NO', COUNTRY_FULL='NORWAY'),
 Row(STN_NO=13840, WBAN=99999, YEARMODA=20190108, TEMP=32.29999923706055, DEWP=26.600000381469727, SLP=9999.900390625, STP=9999.900390625, VISIB=5.199999809265137, WDSP=11.699999809265137, MXSPD=19.399999618530273, GUST=28.0, MAX=None, MIN=None, PRCP=None, SNDP=999.9000244140625, FRSHTT='011000', COUNTRY_ABBR='NO', COUNTRY_FULL='NORWAY'),
 Row(STN_NO=13840, WBAN=99999, YEARMODA=20190109, TEMP=30.200000762939453, DEWP=19.700000762939453, SLP=9999.900390625, STP=9999.900390625, VISIB=999.9000244140625, WDSP=8.199999809265137, MXSPD=19.399999618530273, GUST=999.9000244140625, MAX=None, MIN=None, PRCP=None, SNDP=999.9000244140625, FRSHTT='000000', COUNTRY_ABBR='NO', COUNTRY_FULL='NORWAY'),
 Row(STN_NO=13840, WBAN=99999, YEARMODA=20190110, TEMP=22.899999618530273, DEWP=19.700000762939453, SLP=9999.900390625, STP=9999.900390625, VISIB=14.300000190734863, WDSP=2.5, MXSPD=7.800000190734863, GUST=999.9000244140625, MAX=None, MIN=None, PRCP=None, SNDP=999.9000244140625, FRSHTT='000000', COUNTRY_ABBR='NO', COUNTRY_FULL='NORWAY')]

In [12]:
# filter out null values
# I just filter the columns I will need for answering the questions
df = df.filter( (df.TEMP < 9999) &
                (df.WDSP < 999) 
        )
# select columns for questions
cols = ["YEARMODA", "COUNTRY_FULL", "TEMP", "WDSP", "FRSHTT"]

df = df.select(cols)

In [13]:
df.take(10)

Out[129]: [Row(YEARMODA=20190101, COUNTRY_FULL='NORWAY', TEMP=38.599998474121094, WDSP=11.699999809265137, FRSHTT='000000'),
 Row(YEARMODA=20190102, COUNTRY_FULL='NORWAY', TEMP=30.5, WDSP=8.0, FRSHTT='000000'),
 Row(YEARMODA=20190103, COUNTRY_FULL='NORWAY', TEMP=32.20000076293945, WDSP=3.5, FRSHTT='000000'),
 Row(YEARMODA=20190104, COUNTRY_FULL='NORWAY', TEMP=34.20000076293945, WDSP=2.5999999046325684, FRSHTT='000000'),
 Row(YEARMODA=20190105, COUNTRY_FULL='NORWAY', TEMP=30.600000381469727, WDSP=2.0, FRSHTT='100000'),
 Row(YEARMODA=20190106, COUNTRY_FULL='NORWAY', TEMP=26.600000381469727, WDSP=0.699999988079071, FRSHTT='100000'),
 Row(YEARMODA=20190107, COUNTRY_FULL='NORWAY', TEMP=26.200000762939453, WDSP=1.7999999523162842, FRSHTT='101000'),
 Row(YEARMODA=20190108, COUNTRY_FULL='NORWAY', TEMP=32.29999923706055, WDSP=11.699999809265137, FRSHTT='011000'),
 Row(YEARMODA=20190109, COUNTRY_FULL='NORWAY', TEMP=30.200000762939453, WDSP=8.199999809265137, FRSHTT='000000'),
 Row(YEARMODA=20190110, COUNTRY_FULL='NORWAY', TEMP=22.899999618530273, WDSP=2.5, FRSHTT='000000')]

In [14]:
# QUESTION 1
# Which country had the hottest average mean temperature over the year?

hottest_df = (
    df.groupBy("COUNTRY_FULL")
    .agg(max("TEMP").alias("HOTTEST_AVG_MEAN_TEMPERATURE"))
        )



In [15]:
# ANSWER: HOTTEST AVERAGE MEAN TEMPERATURE
hottest_df.sort('HOTTEST_AVG_MEAN_TEMPERATURE', ascending=False).take(10)

Out[131]: [Row(COUNTRY_FULL='ALGERIA', HOTTEST_AVG_MEAN_TEMPERATURE=110.0),
 Row(COUNTRY_FULL='KUWAIT', HOTTEST_AVG_MEAN_TEMPERATURE=110.0),
 Row(COUNTRY_FULL='IRAN', HOTTEST_AVG_MEAN_TEMPERATURE=109.9000015258789),
 Row(COUNTRY_FULL='IRAQ', HOTTEST_AVG_MEAN_TEMPERATURE=109.9000015258789),
 Row(COUNTRY_FULL='TUNISIA', HOTTEST_AVG_MEAN_TEMPERATURE=109.9000015258789),
 Row(COUNTRY_FULL='UNITED STATES', HOTTEST_AVG_MEAN_TEMPERATURE=109.9000015258789),
 Row(COUNTRY_FULL='INDIA', HOTTEST_AVG_MEAN_TEMPERATURE=109.80000305175781),
 Row(COUNTRY_FULL='CHAD', HOTTEST_AVG_MEAN_TEMPERATURE=109.0),
 Row(COUNTRY_FULL='LIBYA', HOTTEST_AVG_MEAN_TEMPERATURE=108.80000305175781),
 Row(COUNTRY_FULL='TURKEY', HOTTEST_AVG_MEAN_TEMPERATURE=108.0)]

In [16]:
# QUESTION 2
# Which country had the most consecutive days of tornadoes/funnel cloud
# formations?

In [17]:
from pyspark.sql.functions import substring 

df_tornado = df.withColumn('tornado', (substring(df.FRSHTT,5, 1)) )

In [18]:
df_tornado = (
    df_tornado.groupBy(["COUNTRY_FULL", "YEARMODA"])
    .agg(max("tornado").alias("tornado"))
        )


In [19]:
df_tornado.take(3)

Out[144]: [Row(COUNTRY_FULL='ALGERIA', YEARMODA=20190111, tornado='0'),
 Row(COUNTRY_FULL='ALGERIA', YEARMODA=20190312, tornado='0'),
 Row(COUNTRY_FULL='ANTARCTICA', YEARMODA=20190102, tornado='0')]

In [20]:
from pyspark.sql import Window 
from pyspark.sql import functions as f
#Windows definition
w1 = Window.partitionBy(df_tornado.COUNTRY_FULL).orderBy(df_tornado.YEARMODA)
w2 = Window.partitionBy(df_tornado.COUNTRY_FULL, df_tornado.tornado).orderBy(df_tornado.YEARMODA)

res = df_tornado.withColumn('grp',f.row_number().over(w1)-f.row_number().over(w2))
#Window definition for streak
w3 = Window.partitionBy(res.COUNTRY_FULL,res.tornado,res.grp).orderBy(res.YEARMODA)
streak_res = res.withColumn('streak_0',f.when(res.tornado == 1,0).otherwise(f.row_number().over(w3))) \
                .withColumn('streak_1',f.when(res.tornado == 0,0).otherwise(f.row_number().over(w3)))
streak_res.take(3)

Out[145]: [Row(COUNTRY_FULL='ARMENIA', YEARMODA=20190101, tornado='0', grp=0, streak_0=1, streak_1=0),
 Row(COUNTRY_FULL='ARMENIA', YEARMODA=20190102, tornado='0', grp=0, streak_0=2, streak_1=0),
 Row(COUNTRY_FULL='ARMENIA', YEARMODA=20190103, tornado='0', grp=0, streak_0=3, streak_1=0)]

In [21]:
streak_res.groupBy("COUNTRY_FULL").agg(max("streak_1").alias("consecutive_tornado_days")).sort('consecutive_tornado_days', ascending=False).show()


+-------------+------------------------+
 COUNTRY_FULL|consecutive_tornado_days|
+-------------+------------------------+
UNITED STATES| 317|
 MALAYSIA| 289|
 INDIA| 259|
 THAILAND| 241|
 COLOMBIA| 225|
 INDONESIA| 215|
 BRAZIL| 174|
 MEXICO| 162|
 RUSSIA| 154|
 CHINA| 135|
 CAMEROON| 127|
 CANADA| 124|
 PHILIPPINES| 115|
 CONGO| 114|
 NIGERIA| 111|
 MALI| 104|
 GUATEMALA| 104|
 KAZAKHSTAN| 101|
 GHANA| 88|
 CHAD| 72|
+-------------+------------------------+
only showing top 20 rows

In [22]:
# QUESTION 3
# Which country had the second highest average mean wind speed over the year?

In [23]:
wind_df = (
    df.groupBy("COUNTRY_FULL")
    .agg(max("WDSP").alias("max_wind_speed"))
        )

In [24]:
wind_df.sort('max_wind_speed', ascending=False).take(10)

Out[139]: [Row(COUNTRY_FULL='UNITED STATES', max_wind_speed=78.5999984741211),
 Row(COUNTRY_FULL='PORTUGAL', max_wind_speed=73.69999694824219),
 Row(COUNTRY_FULL='KAZAKHSTAN', max_wind_speed=72.80000305175781),
 Row(COUNTRY_FULL='ANTARCTICA', max_wind_speed=71.80000305175781),
 Row(COUNTRY_FULL='ICELAND', max_wind_speed=66.80000305175781),
 Row(COUNTRY_FULL='BULGARIA', max_wind_speed=65.5999984741211),
 Row(COUNTRY_FULL='BOSNIA AND HERZEGOVINA', max_wind_speed=63.400001525878906),
 Row(COUNTRY_FULL='UNITED KINGDOM', max_wind_speed=61.20000076293945),
 Row(COUNTRY_FULL='GREENLAND', max_wind_speed=61.0),
 Row(COUNTRY_FULL='SWEDEN', max_wind_speed=60.900001525878906)]